### Import Libraries

In [ ]:
!pip install pandas numpy scikit-learn matplotlib

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

### Load datasets

In [ ]:
academic_df = pd.read_csv("./data/academic.csv")
academic_df.head()

,SchoolDepartment,CourseTitle,Email,RequiredSkill,Professors,GainedSkill,strartingOrAsgnmtDate,Hour
0,Quality Assurance and Risk Mgmt,Add Documents to 2020 Core Committee Page,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,07:51:00 PM
1,Quality Assurance and Risk Mgmt,White River High School Election Video,kfoulstone0@instagram.com,Video Editing and Production,Fatima Zahra El Idrissi,04-03-2019,07:56:00 PM,NaN
2,Quality Assurance and Risk Manag,Pet License Roster Web Edits,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,08:12:00 PM
3,Quality Assurance and Risk Manag,FJC Shred Event Flyer,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,"Video Editing, Graphic Design, Technical Docum...",2019-03-04,22:27:00
4,Quality Assurance & Risk Management,Create Artwork for new Large Dropboxes,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,11-03-2019,15:38:00,NaN


In [ ]:
department_df = pd.read_csv("./data/department.csv")
department_df.head()

,name,description,department_code,created_date,phone_number,email
0,Quality Assurance and Risk Management,department that is responsible for ensuring th...,dept_01JD8RJE3NTQPR3J6FGQ27DZ9Y,1968-12-28 09:24:59,2126442882953,wgaunter0@meetup.com
1,Technical Communication and Media,department that teaches you how to communicate...,dept_01JD8RJE3XDXME60264HHEEP5X,1929-05-29 14:51:03,2121135897138,wadshede1@feedburner.com
2,Engineering Leadership and Strategy,department that is responsible for the develop...,dept_01JD8RJE3YZMB79PYNABBQDE9V,1937-03-30 22:13:39,2123731667078,ovanderbeken2@jigsy.com
3,Academic Affairs and Staff Development,department that is responsible for the academi...,dept_01JD8RJE3ZVC8A59TW7Q5PQ3K1,1971-06-04 00:51:39,2124593511982,kdoxsey3@qq.com
4,Environmental and Civil Engineering,department that is responsible for the design ...,dept_01JD8RJE40JJA5J75GDEJAJW5R,1920-12-08 09:16:51,2127308767779,fassiter4@nytimes.com


### Clean and Standardize column names

In [ ]:
academic_df.columns = academic_df.columns.str.lower().str.replace(' ', '_')
department_df.columns = department_df.columns.str.lower().str.replace(' ', '_')

In [ ]:
academic_df.head()

,schooldepartment,coursetitle,email,requiredskill,professors,gainedskill,strartingorasgnmtdate,hour
0,Quality Assurance and Risk Mgmt,Add Documents to 2020 Core Committee Page,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,07:51:00 PM
1,Quality Assurance and Risk Mgmt,White River High School Election Video,kfoulstone0@instagram.com,Video Editing and Production,Fatima Zahra El Idrissi,04-03-2019,07:56:00 PM,NaN
2,Quality Assurance and Risk Manag,Pet License Roster Web Edits,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,08:12:00 PM
3,Quality Assurance and Risk Manag,FJC Shred Event Flyer,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,"Video Editing, Graphic Design, Technical Docum...",2019-03-04,22:27:00
4,Quality Assurance & Risk Management,Create Artwork for new Large Dropboxes,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,11-03-2019,15:38:00,NaN


In [ ]:
# since we got a non understandable column name we rename some specific columns
academic_df.rename(columns={'schooldepartment': 'school_department'}, inplace=True)
academic_df.rename(columns={'coursetitle': 'course_title'}, inplace=True)
academic_df.rename(columns={'requiredskill': 'required_skill'}, inplace=True)
academic_df.rename(columns={'gainedskill': 'gained_skill'}, inplace=True)
academic_df.rename(columns={'strartingorasgnmtdate': 'starting_or_assignment_date'}, inplace=True)

In [ ]:
academic_df.head()

,school_department,course_title,email,required_skill,professors,gained_skill,starting_or_assignment_date,hour
0,Quality Assurance and Risk Mgmt,Add Documents to 2020 Core Committee Page,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,07:51:00 PM
1,Quality Assurance and Risk Mgmt,White River High School Election Video,kfoulstone0@instagram.com,Video Editing and Production,Fatima Zahra El Idrissi,04-03-2019,07:56:00 PM,NaN
2,Quality Assurance and Risk Manag,Pet License Roster Web Edits,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,08:12:00 PM
3,Quality Assurance and Risk Manag,FJC Shred Event Flyer,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,"Video Editing, Graphic Design, Technical Docum...",2019-03-04,22:27:00
4,Quality Assurance & Risk Management,Create Artwork for new Large Dropboxes,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,11-03-2019,15:38:00,NaN


In [ ]:
department_df.head()

,name,description,department_code,created_date,phone_number,email
0,Quality Assurance and Risk Management,department that is responsible for ensuring th...,dept_01JD8RJE3NTQPR3J6FGQ27DZ9Y,1968-12-28 09:24:59,2126442882953,wgaunter0@meetup.com
1,Technical Communication and Media,department that teaches you how to communicate...,dept_01JD8RJE3XDXME60264HHEEP5X,1929-05-29 14:51:03,2121135897138,wadshede1@feedburner.com
2,Engineering Leadership and Strategy,department that is responsible for the develop...,dept_01JD8RJE3YZMB79PYNABBQDE9V,1937-03-30 22:13:39,2123731667078,ovanderbeken2@jigsy.com
3,Academic Affairs and Staff Development,department that is responsible for the academi...,dept_01JD8RJE3ZVC8A59TW7Q5PQ3K1,1971-06-04 00:51:39,2124593511982,kdoxsey3@qq.com
4,Environmental and Civil Engineering,department that is responsible for the design ...,dept_01JD8RJE40JJA5J75GDEJAJW5R,1920-12-08 09:16:51,2127308767779,fassiter4@nytimes.com


### Handle Missing Values

In [ ]:
### Let's check first how much missed values are in our dataframes
print("\nMissing values in Academic Dataset:")
print(academic_df.isnull().sum())
print("\nMissing values in Department Dataset:")
print(department_df.isnull().sum())


Missing values in Academic Dataset:
school_department                1
course_title                     2
email                            0
required_skill                   0
professors                       4
gained_skill                     2
starting_or_assignment_date     16
hour                           192
dtype: int64

Missing values in Department Dataset:
name               0
description        0
department_code    0
created_date       0
phone_number       0
email              0
dtype: int64


In [ ]:
academic_df

,school_department,course_title,email,required_skill,professors,gained_skill,starting_or_assignment_date,hour
0,Quality Assurance and Risk Mgmt,Add Documents to 2020 Core Committee Page,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,07:51:00 PM
1,Quality Assurance and Risk Mgmt,White River High School Election Video,kfoulstone0@instagram.com,Video Editing and Production,Fatima Zahra El Idrissi,04-03-2019,07:56:00 PM,NaN
2,Quality Assurance and Risk Manag,Pet License Roster Web Edits,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,08:12:00 PM
3,Quality Assurance and Risk Manag,FJC Shred Event Flyer,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,"Video Editing, Graphic Design, Technical Docum...",2019-03-04,22:27:00
4,Quality Assurance & Risk Management,Create Artwork for new Large Dropboxes,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,11-03-2019,15:38:00,NaN
...,...,...,...,...,...,...,...,...
2134,NaN,NaN,unvalid,2020-12-10,09:08:00 PM,NaN,NaN,NaN
2135,Financal Engineering,Tyler Award Submission,unvalid,2021-01-15,08:13:00 PM,NaN,NaN,NaN
2136,Urban Planning and Public Infra,Stormwater Manual Update,unvalid,Website Programming,NaN,2021-02-17,23:21:00,NaN
2137,Urban Planning & Public Infrastructure,Stormwater Manual Update,unvalid,Website Programming,NaN,2021-02-17,11:25:00 PM,NaN


In [ ]:
# Impute missing 'hour' column in academic_df
academic_df['hour'] = academic_df['hour'].fillna('00:00:00')

# Fill missing values in other columns by 'unknown'
academic_df.fillna('unknown', inplace=True)
academic_df.fillna('unknown', inplace=True)

In [ ]:
print("\nMissing values in Academic Dataset:")
print(academic_df.isnull().sum())
print("\nMissing values in Department Dataset:")
print(department_df.isnull().sum())
# As we can see we removed successfully all NaN values


Missing values in Academic Dataset:
school_department              0
course_title                   0
email                          0
required_skill                 0
professors                     0
gained_skill                   0
starting_or_assignment_date    0
hour                           0
dtype: int64

Missing values in Department Dataset:
name               0
description        0
department_code    0
created_date       0
phone_number       0
email              0
dtype: int64


In [ ]:
# Standardize department names by making everything in lower case
academic_df['school_department'] = academic_df['school_department'].str.lower().str.strip()
department_df['name'] = department_df['name'].str.lower().str.strip()

# Handle special characters in department name like that & become "and"
academic_df['school_department'] = academic_df['school_department'].str.replace('&', 'and')
academic_df[:1]


,school_department,course_title,email,required_skill,professors,gained_skill,starting_or_assignment_date,hour
0,quality assurance and risk mgmt,Add Documents to 2020 Core Committee Page,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,07:51:00 PM


In [ ]:
academic_df

,school_department,course_title,email,required_skill,professors,gained_skill,starting_or_assignment_date,hour
0,quality assurance and risk mgmt,Add Documents to 2020 Core Committee Page,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,07:51:00 PM
1,quality assurance and risk mgmt,White River High School Election Video,kfoulstone0@instagram.com,Video Editing and Production,Fatima Zahra El Idrissi,04-03-2019,07:56:00 PM,00:00:00
2,quality assurance and risk manag,Pet License Roster Web Edits,kfoulstone0@instagram.com,Website Programming,Fatima Zahra El Idrissi,Website Programming,04-03-2019,08:12:00 PM
3,quality assurance and risk manag,FJC Shred Event Flyer,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,"Video Editing, Graphic Design, Technical Docum...",2019-03-04,22:27:00
4,quality assurance and risk management,Create Artwork for new Large Dropboxes,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,11-03-2019,15:38:00,00:00:00
...,...,...,...,...,...,...,...,...
2134,unknown,unknown,unvalid,2020-12-10,09:08:00 PM,unknown,unknown,00:00:00
2135,financal engineering,Tyler Award Submission,unvalid,2021-01-15,08:13:00 PM,unknown,unknown,00:00:00
2136,urban planning and public infra,Stormwater Manual Update,unvalid,Website Programming,unknown,2021-02-17,23:21:00,00:00:00
2137,urban planning and public infrastructure,Stormwater Manual Update,unvalid,Website Programming,unknown,2021-02-17,11:25:00 PM,00:00:00


In [ ]:
# Merge datasets on a common column "school_department"
merged_df = pd.merge(academic_df, department_df, left_on='school_department', right_on='name', how='inner')
merged_df.head()

,school_department,course_title,email_x,required_skill,professors,gained_skill,starting_or_assignment_date,hour,name,description,department_code,created_date,phone_number,email_y
0,quality assurance and risk management,Create Artwork for new Large Dropboxes,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,11-03-2019,15:38:00,00:00:00,quality assurance and risk management,department that is responsible for ensuring th...,dept_01JD8RJE3NTQPR3J6FGQ27DZ9Y,1968-12-28 09:24:59,2126442882953,wgaunter0@meetup.com
1,quality assurance and risk management,Redesign Election Inserts,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,"Graphic Design, Proofreading and Quality Assur...",15-03-2019,07:26:00 PM,quality assurance and risk management,department that is responsible for ensuring th...,dept_01JD8RJE3NTQPR3J6FGQ27DZ9Y,1968-12-28 09:24:59,2126442882953,wgaunter0@meetup.com
2,quality assurance and risk management,Kennel Brochure,kfoulstone0@instagram.com,"Graphic Design/Marketing (Posters, Brochures, ...",Fatima Zahra El Idrissi,"Graphic Design, Proofreading and Quality Assur...",2019-03-15,07:40:00 PM,quality assurance and risk management,department that is responsible for ensuring th...,dept_01JD8RJE3NTQPR3J6FGQ27DZ9Y,1968-12-28 09:24:59,2126442882953,wgaunter0@meetup.com
3,academic affairs and staff development,LinkedIn Learning employee video,kfoulstone0@instagram.com,Video Editing and Production,Fatima Zahra El Idrissi,Video Editing and Production,26-03-2019,10:07:00 PM,academic affairs and staff development,department that is responsible for the academi...,dept_01JD8RJE3ZVC8A59TW7Q5PQ3K1,1971-06-04 00:51:39,2124593511982,kdoxsey3@qq.com
4,technical communication and media,Social Planning,kfoulstone0@instagram.com,Social Media Strategy,Fatima Zahra El Idrissi,Social Media Strategy,2019-04-16,07:04:00 PM,technical communication and media,department that teaches you how to communicate...,dept_01JD8RJE3XDXME60264HHEEP5X,1929-05-29 14:51:03,2121135897138,wadshede1@feedburner.com


In [ ]:
# Combine required and gained skills
merged_df['skills_combined'] = merged_df['required_skill'] + " " + merged_df['gained_skill']

# Encode text features using TF-IDF
tfidf = TfidfVectorizer(max_features=100)
skills_tfidf = tfidf.fit_transform(merged_df['skills_combined']).toarray()

# Add TF-IDF features back to the dataframe
skills_df = pd.DataFrame(skills_tfidf, columns=[f'skill_tfidf_{i}' for i in range(skills_tfidf.shape[1])])
merged_df = pd.concat([merged_df.reset_index(drop=True), skills_df], axis=1)

# Encode categorical columns
le = LabelEncoder()
merged_df['school_department_encoded'] = le.fit_transform(merged_df['school_department'])
merged_df['professors_encoded'] = le.fit_transform(merged_df['professors'])

# Select numerical features for clustering
numerical_features = merged_df.select_dtypes(include=['float64', 'int64']).dropna(axis=1)

# Reduce dimensionality for clustering
pca = PCA(n_components=2)
reduced_features = pca.fit_transform(numerical_features)

# Define clustering models
models = {
    "KMeans": KMeans(n_clusters=5, random_state=42),
    "AgglomerativeClustering": AgglomerativeClustering(n_clusters=5),
    "DBSCAN": DBSCAN(eps=1.0, min_samples=5),
    "GaussianMixture": GaussianMixture(n_components=5, random_state=42)
}

# Evaluate and save models
results = []
for name, model in models.items():
    # Fit or predict the clustering
    if name == "GaussianMixture":
        clusters = model.fit_predict(reduced_features)
    else:
        clusters = model.fit(reduced_features).fit_predict(reduced_features)

    # Save the model
    with open(f"notebook_models/{name}_model.pkl", "wb") as file:
        pickle.dump(model, file)

    # Evaluate performance
    silhouette_avg = silhouette_score(reduced_features, clusters) if len(set(clusters)) > 1 else np.nan
    calinski_harabasz = calinski_harabasz_score(reduced_features, clusters) if len(set(clusters)) > 1 else np.nan
    davies_bouldin = davies_bouldin_score(reduced_features, clusters) if len(set(clusters)) > 1 else np.nan

    # Append results
    results.append({
        "Model": name,
        "Silhouette Score": silhouette_avg,
        "Calinski-Harabasz Index": calinski_harabasz,
        "Davies-Bouldin Index": davies_bouldin
    })

# Print results
results_df = pd.DataFrame(results)
print(results_df)

# Save results to CSV for further analysis
results_df.to_csv("clustering_results.csv", index=False)

                     Model  Silhouette Score  Calinski-Harabasz Index  \
0                   KMeans          0.845880              8129.858810   
1  AgglomerativeClustering          0.841403              8393.571222   
2                   DBSCAN          0.145420               157.156015   
3          GaussianMixture          0.839422              7505.135039   

   Davies-Bouldin Index  
0              0.220000  
1              0.250760  
2              1.734805  
3              0.227394  
